In [1]:
import pandas as pd
import numpy as np
from datetime import time
import pytz

IST = pytz.timezone("Asia/Kolkata")
MARKET_CLOSE = time(15, 30)


In [2]:
news_data = {
    "headline": [
        "Apple Product", "Tesla Earnings", "Amazon Cloud",
        "Microsoft AI", "Google Quarter", "Weekend Apple", "Sat Tesla"
    ],
    "timestamp_utc": [
        "2024-01-15 08:00:00", "2024-01-15 06:00:00", "2024-01-15 14:00:00",
        "2024-01-15 18:30:00", "2024-01-16 01:00:00",
        "2024-01-20 10:00:00", "2024-01-21 12:00:00"
    ],
    "sentiment_score": [0.8, 0.6, 0.7, 0.5, 0.9, 0.4, 0.3]
}

df_news = pd.DataFrame(news_data)
df_news["timestamp_utc"] = pd.to_datetime(df_news["timestamp_utc"], utc=True)

print("Input UTC Data Ready.")


Input UTC Data Ready.


In [3]:
def map_to_trading_day(ts):
    """
    Maps a timezone-aware IST timestamp to the correct trading day
    to avoid look-ahead bias.
    """

    # Weekend → Monday
    if ts.weekday() >= 5:
        return ts + pd.tseries.offsets.BusinessDay(1)

    # After market close → next business day
    if ts.time() > MARKET_CLOSE:
        return ts + pd.tseries.offsets.BusinessDay(1)

    # During market hours → same day
    return ts


In [4]:
df_news["timestamp_ist"] = df_news["timestamp_utc"].dt.tz_convert(IST)
df_news["trading_day"] = df_news["timestamp_ist"].apply(map_to_trading_day)

# Normalize to clean date (no time, no timezone)
df_news["trading_day"] = df_news["trading_day"].dt.normalize()

df_news[["headline", "timestamp_ist", "trading_day"]]


,headline,timestamp_ist,trading_day
0,Apple Product,2024-01-15 13:30:00+05:30,2024-01-15 00:00:00+05:30
1,Tesla Earnings,2024-01-15 11:30:00+05:30,2024-01-15 00:00:00+05:30
2,Amazon Cloud,2024-01-15 19:30:00+05:30,2024-01-16 00:00:00+05:30
3,Microsoft AI,2024-01-16 00:00:00+05:30,2024-01-16 00:00:00+05:30
4,Google Quarter,2024-01-16 06:30:00+05:30,2024-01-16 00:00:00+05:30
5,Weekend Apple,2024-01-20 15:30:00+05:30,2024-01-22 00:00:00+05:30
6,Sat Tesla,2024-01-21 17:30:00+05:30,2024-01-22 00:00:00+05:30


In [5]:
trading_dates = pd.date_range(
    start="2024-01-15",
    end="2024-01-25",
    freq="B"
)

df_stock = pd.DataFrame({
    "date": trading_dates,
    "close": np.random.uniform(150, 160, len(trading_dates))
})


In [6]:
news_agg = (
    df_news
    .groupby("trading_day")
    .agg(
        avg_sentiment=("sentiment_score", "mean"),
        news_count=("sentiment_score", "count")
    )
    .reset_index()
    .rename(columns={"trading_day": "date"})
)

news_agg


,date,avg_sentiment,news_count
0,2024-01-15 00:00:00+05:30,0.70,2
1,2024-01-16 00:00:00+05:30,0.70,3
2,2024-01-22 00:00:00+05:30,0.35,2


In [8]:
# Ensure both merge keys are timezone-naive
news_agg["date"] = news_agg["date"].dt.tz_localize(None)

df_merged = (
    pd.merge(df_stock, news_agg, on="date", how="left")
    .fillna(0)
)

df_merged.head(10)



,date,close,avg_sentiment,news_count
0,2024-01-15,151.743480,0.70,2.0
1,2024-01-16,159.387430,0.70,3.0
2,2024-01-17,156.069926,0.00,0.0
3,2024-01-18,150.092253,0.00,0.0
4,2024-01-19,159.205521,0.00,0.0
5,2024-01-22,150.240192,0.35,2.0
6,2024-01-23,156.784335,0.00,0.0
7,2024-01-24,155.584745,0.00,0.0
8,2024-01-25,158.719642,0.00,0.0
